In [16]:
from evaluate import load
from evaluate.module import EvaluationModule
from datasets import load_dataset, Dataset
from models.qg import QG
import torch

qg = QG("the-coorporation/t5-small-qg-2.0", "the-coorporation/t5-small-qg-2.0")

## BLEU

In [17]:
bleu: EvaluationModule = load("bleu")

## METEOR

In [18]:
meteor: EvaluationModule = load("meteor")

[nltk_data] Downloading package wordnet to /home/laugu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/laugu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/laugu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [24]:
squad: Dataset = load_dataset("the-coorporation/the_squad_qg")
validation = squad['validation']
validation

No config specified, defaulting to: the_squad_qg/v2
Found cached dataset the_squad_qg (/home/laugu/.cache/huggingface/datasets/the-coorporation___the_squad_qg/v2/2.0.0/cd5e3386e9e3124c24114539fb5124093fa58d99c1a7292451b8801a94a2aca5)


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['context', 'questions'],
    num_rows: 1204
})

In [22]:
references: list[dict] = [entry['questions'].replace(" {sep_token}", "") for entry in validation]

predictions: list[dict] = []

device = "cuda" if torch.cuda.is_available() else "cpu"

generator_args = {
    "max_length": 512,
    "num_beams": 4,
    "length_penalty": 1.5,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
}


for i, entry in enumerate(validation):
    if i == 1:
        break
    
    context: str = entry['context']
    input_string = "generate questions: " + context + " </s>"

    input_ids = qg._tokenizer.encode(input_string, return_tensors="pt").to(device)
    result = qg._model.generate(input_ids, **generator_args).to(device)
    questions = qg._tokenizer.decode(result[0], skip_special_tokens=True)
    questions

In [23]:
# p = predictions
# r = references
# flat_list = [item for sublist in nested_list for item in sublist]
# p = [question for questions in predictions['questions'] for question in questions]
# p = [questions for questions in predictions['questions'] for prediction in predictions]
# p = [question for question in predictions['questions']]
p = [question for prediction in predictions for question in prediction['questions']]
r = [question for reference in references for question in reference['questions']]
rouge.compute(references=r, predictions=p)

TypeError: string indices must be integers